<a href="https://colab.research.google.com/github/masies/CRA/blob/main/pytorchConversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip3 install transformers
!pip install sentencepiece==0.1.94
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -q transformers==3.5.0 fast-trees
!git clone -q https://github.com/microsoft/CodeXGLUE.git


import pandas as pd
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
import torch
from tqdm import tqdm
import os
import logging
import shutil
from torch.utils.data import Dataset, DataLoader
import json
import sys

from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'

!gcloud config set project {project_id}

!mkdir core

!gsutil -m cp -r \
  "gs://code_review_automation/pyTorch_coversion/core/Logger.py" \
  ./core

from core.Logger import Logger

# script for conversion in pythorch
!gsutil cp gs://{bucket_name}/pyTorch_coversion/code/tf_2_pytorch_T5.py ./tf_2_pytorch_T5.py

# Download the configuration file
!gsutil cp gs://{bucket_name}/pyTorch_coversion/config/config.json ./current_model/config.json


     |████████████████████████████████| 1.1MB 8.7MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3MB 9.2MB/s 
     |████████████████████████████████| 2.9MB 38.1MB/s 
     |████████████████████████████████| 1.1MB 49.6MB/s 
     |████████████████████████████████| 870kB 45.3MB/s 
     |████████████████████████████████| 163kB 46.1MB/s 
     |████████████████████████████████| 112kB 51.4MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
Updated property [core/project].
Copying gs://code_review_automation/pyTorch_coversion/core/Logger.py...
/ [1/1 files][   1016 B/   1016 B] 100% Done                                    
Operation completed over 1 objects/1016.0 B.                                     
Copying gs://cod

In [2]:
model_number = 240000

# Download the selected best model
!mkdir dumps
!mkdir current_model
!gsutil -m cp \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.data-00000-of-00002" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.data-00001-of-00002" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.index" \
  "gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-{model_number}.meta" \
  ./current_model/

# Convert the model
!python3 ./tf_2_pytorch_T5.py --tf_checkpoint_path ./current_model/model.ckpt-{model_number} --config_file ./current_model/config.json --pytorch_dump_path ./dumps

mkdir: cannot create directory ‘current_model’: File exists
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.data-00000-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.data-00001-of-00002...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.index...
Copying gs://code_review_automation/fine_tuning/HP_tuning/final_version/comment_v1/codeANDcomment_code/pretrained/model.ckpt-240000.meta...
\ [4/4 files][126.0 MiB/126.0 MiB] 100% Done                                    
Operation completed over 4 objects/126.0 MiB.                                    
2021-04-10 11:55:42.594373: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Building PyTorch model from configuratio

In [7]:
# Download the validation script
!gsutil cp gs://{bucket_name}/pyTorch_coversion/code/validation.py ./validation.py


# Download the model and vocab
!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.model ./current_model/TestModel.model
!gsutil cp gs://{bucket_name}/CodeReviewModel/TestModel.vocab ./current_model/TestModel.vocab

# download the test set
!gsutil cp gs://{bucket_name}/dataset/old/fineTuningDataset_v1/codeANDcomment_code/test.tsv ./data/codeANDcomment_code/test.tsv

# prepare source and target files
df = pd.read_csv("./data/codeANDcomment_code/test.tsv", sep='\t', names=["source","target"])

# initialize source and target files
f = open("./data/codeANDcomment_code/test.source", "w")
f.close()
f = open("./data/codeANDcomment_code/test.target", "w")

f.close()
with open("./data/codeANDcomment_code/test.source", "a") as source:
  with open("./data/codeANDcomment_code/test.target", "a") as target:
    for index, row in df.iterrows():
      if index < 20:
        source.write("code&comment2code: " + row.source + "\n")
        target.write(row.target + "\n")

Copying gs://code_review_automation/pyTorch_coversion/code/validation.py...
/ [1 files][  6.1 KiB/  6.1 KiB]                                                
Operation completed over 1 objects/6.1 KiB.                                      
Copying gs://code_review_automation/CodeReviewModel/TestModel.model...
/ [1 files][762.0 KiB/762.0 KiB]                                                
Operation completed over 1 objects/762.0 KiB.                                    
Copying gs://code_review_automation/CodeReviewModel/TestModel.vocab...
/ [1 files][557.4 KiB/557.4 KiB]                                                
Operation completed over 1 objects/557.4 KiB.                                    
Copying gs://code_review_automation/dataset/old/fineTuningDataset_v1/codeANDcomment_code/test.tsv...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      


In [8]:
class EvalDataset(torch.utils.data.Dataset):
  samples = []

  def __init__(self,  data_dir_path):
    data_dir = data_dir_path
    datasets = ['test.source', 'test.target']
    self.samples = []    

    input_file = open(os.path.join(data_dir, 'test.source'), 'r')
    output_file = open(os.path.join(data_dir, 'test.target'), 'r')

    lines_input = input_file.readlines()
    output_lines = output_file.readlines()

    for (inp, out) in zip(lines_input, output_lines):
      self.samples.append((inp.rstrip(), out.rstrip()))

  def __getitem__(self, index):
    return self.samples[index]

  def __len__(self):
    return len(self.samples)

In [9]:
beam_size = 3
batch_size = 64
data_dir = "/content/data/codeANDcomment_code"
tokenizer_name = "./current_model/TestModel.model" 
model_name_or_path = "./dumps/pytorch_model.bin"
config_name = "./current_model/config.json"

dataset = EvalDataset(data_dir)
dloader = DataLoader(dataset=dataset, batch_size=batch_size, num_workers=0)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # My envirnment uses CPU

t5_tokenizer = T5Tokenizer.from_pretrained(tokenizer_name)

t5_config = T5Config.from_pretrained(config_name)
t5_mlm = T5ForConditionalGeneration.from_pretrained(model_name_or_path, config=t5_config).to(DEVICE)
        
# t5_mlm.eval()

Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated


In [10]:
predictions = []
code_bleues = []

perfect = 0

# indexes for batches
old = 0
new = batch_size * beam_size

for batch in tqdm(dloader): 
  encoded = t5_tokenizer.batch_encode_plus(batch[0], add_special_tokens=False, return_tensors='pt', padding=True)
  
  input_ids = encoded['input_ids'].to(DEVICE)
  attention_mask = encoded['attention_mask'].to(DEVICE)

  outputs = t5_mlm.generate(
      input_ids=input_ids,
      max_length=512, #Change here 
      num_beams=beam_size,
      attention_mask=attention_mask,
      early_stopping=True,
      num_return_sequences=beam_size).to(DEVICE)

  predictions.extend(t5_tokenizer.batch_decode(outputs, skip_special_tokens=True))

  to_analyze = predictions[old:new]
  target_list = batch[1]
  input_list = batch[0]

  idx = 0
  for (input_item, target_item) in zip(input_list,target_list):

    flag_perfect = False
    target_item = "".join(target_item.split(' '))
    best_code_bleu = 0
    
    for i in range(beam_size):
      prediction_item = "".join(to_analyze[idx].split(' '))

      if not flag_perfect and prediction_item == target_item:
        perfect += 1
        flag_perfect = True
      
      idx += 1

      with open("code_bleu_target.txt", "w") as target_cb:
        target_cb.write(target_item + "\n")
      with open("code_bleu_prediction.txt", "w") as prediction_cb:
        prediction_cb.write(prediction_item + "\n")

      bleu_computed = False

      try:
        result = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../code_bleu_target.txt --hyp ./../../../../../code_bleu_prediction.txt --lang java --params 0.25,0.25,0.25,0.25
        code_bleu = float(result[1][result[1].index("CodeBLEU score:  ")+17:])
        best_code_bleu = code_bleu if code_bleu > best_code_bleu else best_code_bleu
        bleu_computed = True
      
      except: 
        print("problem in computing code bleu")
        # print(result)
        # print(target_item)
        # print(prediction_item)
    if bleu_computed:
      code_bleues.append(best_code_bleu)

  old = new
  new = new + (batch_size * beam_size)

code_bleues
print()
print('#perfect prediction: ', perfect)
print('Perfect prediction {}'.format((perfect/len(dataset))*100))
print('Average Bleu {}'.format(sum(code_bleues) / len(code_bleues)))
print('len Bleues {}'.format(len(code_bleues)))




  0%|          | 0/1 [00:00<?, ?it/s]

problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu
problem in computing code bleu


100%|██████████| 1/1 [00:20<00:00, 20.50s/it]


#perfect prediction:  7
Perfect prediction 35.0
Average Bleu 0.4542773641485573
len Bleues 15


In [96]:
code = "public static void main(String[] args) { \u003CSTART> System.out.println(\" is vowel\"); else System.out.println(ch + \" is consonant\"); \u003CEND>}"#@param {type:"string"}
comment = "use a logger here" #@param {type:"string"}
input = "code&comment2code: <code>" + code + "</code><technical_language>" + comment + "</technical_language>"
encoded = t5_tokenizer.encode(input, add_special_tokens=False, return_tensors='pt', padding=True).to(DEVICE)


input_ids = encoded
outputs = t5_mlm.generate(
      input_ids=input_ids,
      max_length=512, #Change here 
      num_beams=1,
      early_stopping=True,
      num_return_sequences=1).to(DEVICE)

print(code)
print(t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


public static void main(String[] args) { <START> System.out.println(" is vowel"); else System.out.println(ch + " is consonant"); <END>}
public static void main(String[] args) { logger.info(" is vowel"); else logger.info(ch + " is consonant"); }


In [63]:
# run validation script
!python3 validation.py --beam_size 3 --batch_size 10 --data_dir /content/data/codeANDcomment_code --tokenizer_name ./current_model/TestModel.model --model_name_or_path ./dumps/pytorch_model.bin --config_name ./current_model/config.json


2021-04-09 15:20:56.282517: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated
100% 2/2 [00:09<00:00,  4.88s/it]
#perfect prediction:  7
Perfect prediction 35.0


In [30]:
# from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration
# t5_tokenizer = T5Tokenizer.from_pretrained("./current_model/TestModel.model")
# tokens = t5_tokenizer.encode("public static main(String args){ Int a = 4}")
# print(len(tokens))
# t5_tokenizer.convert_ids_to_tokens(tokens)

13


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1621: FutureWarning: Calling T5Tokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


['▁public',
 '▁static',
 '▁main',
 '(',
 'String',
 '▁args',
 '){',
 '▁Int',
 '▁a',
 '▁=',
 '▁4',
 '}',
 '</s>']